In [11]:
import tkinter
from tkinter import *
#to display image in GUI 
from PIL import Image
from PIL import ImageTk
#allows us to browse our filesystme 
from tkinter import filedialog
import cv2
import numpy as np

In [12]:
def open_original_image():
    global panelA
    global Ocanvas
    global Opath
    #navigate our filesystem 
    Opath=filedialog.askopenfilename()
    
    if len(Opath)>0:
        image=cv2.imread(Opath,1)
        image= cv2.resize(image, (400,400)) 
        # images in RGB order, so we need to swap the channels
        image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        #convert images to PIL format 
        image=Image.fromarray(image)
        #convert to image tkinter format
        image=ImageTk.PhotoImage(image)

        
        if panelA is None:
            
            panelA=Ocanvas
            panelA.image=image
            panelA.create_image(0,0,image=image,anchor=NW)
            #panelA.pack()
            
        else:
            print("beeb")
            panelA.delete("all")
            panelA.image=image
            panelA.configure(image)
            
def open_manufactured_image():
    global panelB
    global Mcanvas
    global Mpath
    #navigate our filesystem 
    Mpath=filedialog.askopenfilename()
    
    if len(Mpath)>0:
        image=cv2.imread(Mpath,1)
        image= cv2.resize(image, (400,400))
        # images in RGB order, so we need to swap the channels
        image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        #convert images to PIL format 
        image=Image.fromarray(image)
        #convert to image tkinter format
        image=ImageTk.PhotoImage(image)
        
        if panelB is None:
            #panelA=tkinter.Canvas(width=400,height=400,bg="white")
            panelB=Mcanvas
            panelB.image=image
            panelB.create_image(0,0,image=image,anchor=NW)
            panelB.pack()
            
        else:
            panelB.configure(image=image)
            panelB.image=image
    
def display_Result_image():
    global panelC
    global Rcanvas
    global ResultImage
    ResultImage=cv2.resize(ResultImage, (400,400))
    ResultImage=cv2.cvtColor(ResultImage,cv2.COLOR_BGR2RGB)
    ResultImage=Image.fromarray(ResultImage)
    ResultImage=ImageTk.PhotoImage(ResultImage)

    if panelC is None:
        #panelA=tkinter.Canvas(width=400,height=400,bg="white")
        panelC=Rcanvas
        panelC.image=ResultImage
        panelC.create_image(0,0,image=ResultImage,anchor=NW)
        panelC.pack()

    else:
        panelC.configure(image=ResultImage)
        panelC.image=ResultImage    
    
def image_processing():
    global ResultImage
    OriginalImg=cv2.imread(Opath,1)
    TestImage=cv2.imread(Mpath,1)
    
    orig = cv2.cvtColor(OriginalImg,cv2.COLOR_RGB2GRAY)
    test_gray = cv2.cvtColor(TestImage,cv2.COLOR_RGB2GRAY)
    
    canny_1 = cv2.Canny(orig, 120, 120 )
    canny_2 = cv2.Canny(test_gray, 120, 120 )
    
    subtracted = cv2.subtract(canny_1 , canny_2)
    
    blur = cv2.GaussianBlur(subtracted,(11,11),0)
    x = blur*-0.75
    x = np.uint8(x)
    x = cv2.cvtColor(x, cv2.COLOR_GRAY2BGR)
    x = cv2.cvtColor(x, cv2.COLOR_BGR2HSV)
    x[:,:,0] = 5
    x = cv2.cvtColor(x, cv2.COLOR_HSV2BGR)
    detector = cv2.SimpleBlobDetector_create()
    # Detect blobs.
    keypoints = detector.detect(x)

    # Draw detected blobs as red circles.
    # cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS ensures the size of
    # the circle corresponds to the size of blob
    blank = np.zeros((1,1)) 
    blobs = cv2.drawKeypoints(x, keypoints, blank, (0,255,255),
                                          cv2.DRAW_MATCHES_FLAGS_DEFAULT)
    # plt.subplot(121),plt.imshow(image_1,cmap = 'gray'
    ResultImage=TestImage+x
    display_Result_image()

In [ ]:
root=Tk()
panelA=None
panelB=None
panelC=None
#create check button frame
BtnFrame=tkinter.Frame(root)
BtnFrame.pack(side=TOP,fill=X,pady=5)
CheckBtn=tkinter.Button(BtnFrame,text="Check",command=image_processing)
CheckBtn.pack(fill=X)

#create Original image space
Frame1=tkinter.Frame(root)
Frame1.pack(side=LEFT,fill=Y,padx=2)

#original image label
Olabel=tkinter.Label(Frame1,text="original image")
Olabel.pack(side=TOP,fill=X)

#choose image button
ChooseBtn=tkinter.Button(Frame1,text="Choose image",command=open_original_image)
ChooseBtn.pack(fill=X)

#canvas that contains image
Ocanvas=tkinter.Canvas(Frame1,width=400,height=400,bg="white")
Ocanvas.pack()

#''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''

#creat Manufactured image space
Frame2=tkinter.Frame(root)
Frame2.pack(side=LEFT,padx=2)

#Manufactured image label
Mlabel=tkinter.Label(Frame2,text="Manufatured image")
Mlabel.pack(side=TOP,fill=X)

#choose image button
ChooseBtn1=tkinter.Button(Frame2,text="Choose image",command=open_manufactured_image)
ChooseBtn1.pack(fill=X)

#canvas that contains image
Mcanvas=tkinter.Canvas(Frame2,width=400,height=400,bg="white")
Mcanvas.pack()

#---------------------------------------------------------------------------------

#create result image space
Frame3=tkinter.Frame(root)
Frame3.pack(side=LEFT,padx=2)

#Result label 
Rlabel=tkinter.Label(Frame3,text="Result image")
Rlabel.pack(fill=X,pady=12)

#canvas that contain this image
Rcanvas=tkinter.Canvas(Frame3,width=400,height=400,bg="white")
Rcanvas.pack()


root.mainloop()